# Setup

This is following [10 minutes to pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) for pandas 1.4.3

In [1]:
import numpy as np
import pandas as pd

# Object creation

In [2]:
# Series - pass a list to pd.Series
s1 = pd.Series([1, 2, 3, np.nan, 5])

In [3]:
s1

0    1.0
1    2.0
2    3.0
3    NaN
4    5.0
dtype: float64

In [4]:
# create an index to use in the dataframe
# default frequency is D (day), so creates a DatetimeIndex of 10 days
dates = pd.date_range('20220619', periods=10)
dates

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

just in case you are wondering, here's [all the "offset aliases"](https://pandas.pydata.org/docs/user_guide/timeseries.html#timeseries-offset-aliases) for Datetime

In [5]:
# Dataframe - pass a Numpy array
# create a 10 row, 4 col random number array, index by dates, give some column names
df = pd.DataFrame(np.random.randn(10, 4), index=dates, columns=list('ABCD'))
df

,A,B,C,D
2022-06-19,0.362133,-1.024067,0.443101,0.537134
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-21,1.209084,0.286537,0.601964,-0.762744
2022-06-22,2.597306,-0.306801,-0.664610,1.789516
2022-06-23,0.467150,0.600479,-0.082923,0.051940
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571
2022-06-25,-0.172528,0.830876,0.912158,1.329058
2022-06-26,0.627848,0.435499,2.178048,1.557753
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831
2022-06-28,-0.436952,1.171574,-0.445353,0.789487


In [6]:
# Create a dataframe by passing a dictionary of objects
# where each object can be converted into a series-like structure
df2 = pd.DataFrame(
    {
        "A": 1.0,
        "B": pd.Timestamp("20220619"),
        "C": pd.Series(1, index=list(range(6)), dtype="float32"),
        "D": np.array([3]*6, dtype="int32"),
        "E": pd.Categorical(["test", "train", "test", "train", "test", "train"]),
        "F": list("foofoo"),
        "G": "foo"
    }
)

df2

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


In [7]:
# the datatype of each of the columns would be different
df2.dtypes

A           float64
B    datetime64[ns]
C           float32
D             int32
E          category
F            object
G            object
dtype: object

# Viewing Data

In [8]:
# top of the dataframe
df.head()

,A,B,C,D
2022-06-19,0.362133,-1.024067,0.443101,0.537134
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-21,1.209084,0.286537,0.601964,-0.762744
2022-06-22,2.597306,-0.306801,-0.664610,1.789516
2022-06-23,0.467150,0.600479,-0.082923,0.051940


In [9]:
# bottom 3 records of the dataframe
df.tail(3)

,A,B,C,D
2022-06-26,0.627848,0.435499,2.178048,1.557753
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831
2022-06-28,-0.436952,1.171574,-0.445353,0.789487


In [10]:
# index of the df
df.index

DatetimeIndex(['2022-06-19', '2022-06-20', '2022-06-21', '2022-06-22',
               '2022-06-23', '2022-06-24', '2022-06-25', '2022-06-26',
               '2022-06-27', '2022-06-28'],
              dtype='datetime64[ns]', freq='D')

In [11]:
# columns of the df
df.columns

Index(['A', 'B', 'C', 'D'], dtype='object')

In [12]:
# .to_numpy gives a NumPy representation of the dataframe
# this is expensive if all columns are of different data type
df.to_numpy()

array([[ 0.36213348, -1.02406689,  0.44310074,  0.53713412],
       [ 0.37108796,  0.72946331, -0.95315613,  1.08166906],
       [ 1.20908399,  0.28653671,  0.60196434, -0.76274353],
       [ 2.59730558, -0.30680111, -0.66461004,  1.78951564],
       [ 0.46715034,  0.6004794 , -0.08292297,  0.05193951],
       [ 1.58307752, -1.85410262, -0.66053243, -0.34257119],
       [-0.17252837,  0.83087571,  0.91215832,  1.32905772],
       [ 0.62784758,  0.43549859,  2.17804755,  1.55775253],
       [-0.55115732,  0.06973498, -0.3286316 , -0.88383086],
       [-0.43695241,  1.17157372, -0.44535283,  0.78948671]])

In [13]:
df2.to_numpy()

array([[1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'f',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'test', 'o',
        'foo'],
       [1.0, Timestamp('2022-06-19 00:00:00'), 1.0, 3, 'train', 'o',
        'foo']], dtype=object)

In [14]:
# quick summary stats
df.describe()

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.605705,0.093919,0.100006,0.514741
std,0.969018,0.926438,0.950692,0.960935
min,-0.551157,-1.854103,-0.953156,-0.883831
25%,-0.038863,-0.212667,-0.606738,-0.243944
50%,0.419119,0.361018,-0.205777,0.663310
75%,1.063775,0.697217,0.562248,1.267211
max,2.597306,1.171574,2.178048,1.789516


1. **count** = Count number of non-NA/null observations
1. **max** = Maximum of the values in the object
1. **min** = Minimum of the values in the object
1. **mean** = Mean of the values
1. **std** = Standard deviation of the observations
1. **25%** = Default lower percentile
1. **50%** = 50 percentile - same as the median 
1. **75%** = Default upper percentile

In [15]:
# change the percentiles
df.describe(percentiles=[.1, .5, .9])

,A,B,C,D
count,10.000000,10.000000,10.000000,10.000000
mean,0.605705,0.093919,0.100006,0.514741
std,0.969018,0.926438,0.950692,0.960935
min,-0.551157,-1.854103,-0.953156,-0.883831
10%,-0.448373,-1.107070,-0.693465,-0.774852
50%,0.419119,0.361018,-0.205777,0.663310
90%,1.684500,0.864946,1.038747,1.580929
max,2.597306,1.171574,2.178048,1.789516


In [16]:
# E, F, G in d2 are not numeric, so do not come up in describe()
df2.describe()

,A,C,D
count,6.0,6.0,6.0
mean,1.0,1.0,3.0
std,0.0,0.0,0.0
min,1.0,1.0,3.0
25%,1.0,1.0,3.0
50%,1.0,1.0,3.0
75%,1.0,1.0,3.0
max,1.0,1.0,3.0


In [17]:
# transpose the data
# turn rows to columns and vice versa
df.T

,2022-06-19,2022-06-20,2022-06-21,2022-06-22,2022-06-23,2022-06-24,2022-06-25,2022-06-26,2022-06-27,2022-06-28
A,0.362133,0.371088,1.209084,2.597306,0.467150,1.583078,-0.172528,0.627848,-0.551157,-0.436952
B,-1.024067,0.729463,0.286537,-0.306801,0.600479,-1.854103,0.830876,0.435499,0.069735,1.171574
C,0.443101,-0.953156,0.601964,-0.664610,-0.082923,-0.660532,0.912158,2.178048,-0.328632,-0.445353
D,0.537134,1.081669,-0.762744,1.789516,0.051940,-0.342571,1.329058,1.557753,-0.883831,0.789487


### [Axis in pandas](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.set_axis.html?highlight=set_ax#pandas.DataFrame.set_axis)

```axis{0 or ‘index’, 1 or ‘columns’}```

Always remember that when you specify:
* ```axis=0``` or ```axis='index'``` you indicate that the operation should be along the _index_ (aka across the rows). More often than not indexes will go from top to bottom (vertically), but sometimes they may not (for e.g. if you use columns as index or when you have hierarchical or multi-index data).
* ```axis=1``` or ```axis='columns'``` indicates the operation is along the columns

When in doubt use the explicit version ```'index'``` or ```'columns'``` instead of ```0``` or ```1```

In [18]:
# sort along the axis - 1 = horizontal
df.sort_index(axis=1, ascending=False)

,D,C,B,A
2022-06-19,0.537134,0.443101,-1.024067,0.362133
2022-06-20,1.081669,-0.953156,0.729463,0.371088
2022-06-21,-0.762744,0.601964,0.286537,1.209084
2022-06-22,1.789516,-0.664610,-0.306801,2.597306
2022-06-23,0.051940,-0.082923,0.600479,0.467150
2022-06-24,-0.342571,-0.660532,-1.854103,1.583078
2022-06-25,1.329058,0.912158,0.830876,-0.172528
2022-06-26,1.557753,2.178048,0.435499,0.627848
2022-06-27,-0.883831,-0.328632,0.069735,-0.551157
2022-06-28,0.789487,-0.445353,1.171574,-0.436952


In [19]:
# sort along the axis - 0 = vertical
df.sort_index(axis=0, ascending=False)

,A,B,C,D
2022-06-28,-0.436952,1.171574,-0.445353,0.789487
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831
2022-06-26,0.627848,0.435499,2.178048,1.557753
2022-06-25,-0.172528,0.830876,0.912158,1.329058
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571
2022-06-23,0.467150,0.600479,-0.082923,0.051940
2022-06-22,2.597306,-0.306801,-0.664610,1.789516
2022-06-21,1.209084,0.286537,0.601964,-0.762744
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-19,0.362133,-1.024067,0.443101,0.537134


In [20]:
# sort ascending by values in a column
df.sort_values(by="A")

,A,B,C,D
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831
2022-06-28,-0.436952,1.171574,-0.445353,0.789487
2022-06-25,-0.172528,0.830876,0.912158,1.329058
2022-06-19,0.362133,-1.024067,0.443101,0.537134
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-23,0.467150,0.600479,-0.082923,0.051940
2022-06-26,0.627848,0.435499,2.178048,1.557753
2022-06-21,1.209084,0.286537,0.601964,-0.762744
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571
2022-06-22,2.597306,-0.306801,-0.664610,1.789516


In [21]:
# sort by non-numerical values
df2.sort_values(by="F", ascending=False)

,A,B,C,D,E,F,G
1,1.0,2022-06-19,1.0,3,train,o,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo


In [22]:
# sort by two or more columns
df2.sort_values(by=["F", "E"])

,A,B,C,D,E,F,G
0,1.0,2022-06-19,1.0,3,test,f,foo
3,1.0,2022-06-19,1.0,3,train,f,foo
2,1.0,2022-06-19,1.0,3,test,o,foo
4,1.0,2022-06-19,1.0,3,test,o,foo
1,1.0,2022-06-19,1.0,3,train,o,foo
5,1.0,2022-06-19,1.0,3,train,o,foo


# Selection

For production prefer the following instead of other data access methods (typical python methods like ["col"] or [a:b] slices etc.):

```.at, .iat, .loc and .iloc.```

## Getting

In [23]:
# selecting a single column returns a Series object
df["A"]

2022-06-19    0.362133
2022-06-20    0.371088
2022-06-21    1.209084
2022-06-22    2.597306
2022-06-23    0.467150
2022-06-24    1.583078
2022-06-25   -0.172528
2022-06-26    0.627848
2022-06-27   -0.551157
2022-06-28   -0.436952
Freq: D, Name: A, dtype: float64

In [24]:
# selecting a slice
df[1:5]

,A,B,C,D
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-21,1.209084,0.286537,0.601964,-0.762744
2022-06-22,2.597306,-0.306801,-0.664610,1.789516
2022-06-23,0.467150,0.600479,-0.082923,0.051940


## Selection by label

In [25]:
# selecting based on a label
df.loc[dates[0]]

A    0.362133
B   -1.024067
C    0.443101
D    0.537134
Name: 2022-06-19 00:00:00, dtype: float64

In [26]:
# select on a multi-axis by lable
# I honestly do not know what the comment above means...help!
df.loc[:, ["A", "B"]]

,A,B
2022-06-19,0.362133,-1.024067
2022-06-20,0.371088,0.729463
2022-06-21,1.209084,0.286537
2022-06-22,2.597306,-0.306801
2022-06-23,0.467150,0.600479
2022-06-24,1.583078,-1.854103
2022-06-25,-0.172528,0.830876
2022-06-26,0.627848,0.435499
2022-06-27,-0.551157,0.069735
2022-06-28,-0.436952,1.171574


In [27]:
# specific index value results in reduction of dimensions
res = df.loc["2022-06-20"]
print(res)
print("res.shape = ", res.shape, " vs. df.shape = ", df.shape)

A    0.371088
B    0.729463
C   -0.953156
D    1.081669
Name: 2022-06-20 00:00:00, dtype: float64
res.shape =  (4,)  vs. df.shape =  (10, 4)


In [28]:
# get to a specific scalar:
#
# method one
df.loc[dates[0], "A"]

0.3621334799110662

In [29]:
#
# method two (slightly faster than method one)
df.at[dates[0], "A"]

0.3621334799110662

## Selection by position

In [30]:
df.iloc[2]

A    1.209084
B    0.286537
C    0.601964
D   -0.762744
Name: 2022-06-21 00:00:00, dtype: float64

In [31]:
# slices - similar to NumPy / Python - [row:slice, col:slice]
df.iloc[1:5, 0:2]

,A,B
2022-06-20,0.371088,0.729463
2022-06-21,1.209084,0.286537
2022-06-22,2.597306,-0.306801
2022-06-23,0.467150,0.600479


In [32]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
df.iloc[[0, 1, 2, 6], [0, 2]]

,A,C
2022-06-19,0.362133,0.443101
2022-06-20,0.371088,-0.953156
2022-06-21,1.209084,0.601964
2022-06-25,-0.172528,0.912158


In [33]:
# by list of locations - similar to NumPy / Python - [[list of rows], [list of cols]]
# change the order of columns, repeact a column
df.iloc[[0, 1, 2, 6], [2, 1, 0, 2]]

,C,B,A,C
2022-06-19,0.443101,-1.024067,0.362133,0.443101
2022-06-20,-0.953156,0.729463,0.371088,-0.953156
2022-06-21,0.601964,0.286537,1.209084,0.601964
2022-06-25,0.912158,0.830876,-0.172528,0.912158


In [34]:
# slice rows explicitly, keep all columns
df.iloc[[1, 2], :]

,A,B,C,D
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-21,1.209084,0.286537,0.601964,-0.762744


In [35]:
# slice columns, keep all rows
df.iloc[:, [2, 3]]

,C,D
2022-06-19,0.443101,0.537134
2022-06-20,-0.953156,1.081669
2022-06-21,0.601964,-0.762744
2022-06-22,-0.664610,1.789516
2022-06-23,-0.082923,0.051940
2022-06-24,-0.660532,-0.342571
2022-06-25,0.912158,1.329058
2022-06-26,2.178048,1.557753
2022-06-27,-0.328632,-0.883831
2022-06-28,-0.445353,0.789487


In [36]:
# everything, because you can
df.iloc[:, :]

,A,B,C,D
2022-06-19,0.362133,-1.024067,0.443101,0.537134
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-21,1.209084,0.286537,0.601964,-0.762744
2022-06-22,2.597306,-0.306801,-0.664610,1.789516
2022-06-23,0.467150,0.600479,-0.082923,0.051940
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571
2022-06-25,-0.172528,0.830876,0.912158,1.329058
2022-06-26,0.627848,0.435499,2.178048,1.557753
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831
2022-06-28,-0.436952,1.171574,-0.445353,0.789487


In [37]:
# get to a scalar (2 methods, just like before)
#
# method one: use iloc
df.iloc[1, 2]

-0.9531561269447512

In [38]:
#
# method two: use iat
df.iat[1, 2]

-0.9531561269447512

## Boolean Indexing

In [39]:
# use a value found in a single col to get data
df[df["A"] > 0]

,A,B,C,D
2022-06-19,0.362133,-1.024067,0.443101,0.537134
2022-06-20,0.371088,0.729463,-0.953156,1.081669
2022-06-21,1.209084,0.286537,0.601964,-0.762744
2022-06-22,2.597306,-0.306801,-0.664610,1.789516
2022-06-23,0.467150,0.600479,-0.082923,0.051940
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571
2022-06-26,0.627848,0.435499,2.178048,1.557753


In [40]:
# boolean across the entire DF - vals that don't match go NaN
df[df > 0]

,A,B,C,D
2022-06-19,0.362133,NaN,0.443101,0.537134
2022-06-20,0.371088,0.729463,NaN,1.081669
2022-06-21,1.209084,0.286537,0.601964,NaN
2022-06-22,2.597306,NaN,NaN,1.789516
2022-06-23,0.467150,0.600479,NaN,0.051940
2022-06-24,1.583078,NaN,NaN,NaN
2022-06-25,NaN,0.830876,0.912158,1.329058
2022-06-26,0.627848,0.435499,2.178048,1.557753
2022-06-27,NaN,0.069735,NaN,NaN
2022-06-28,NaN,1.171574,NaN,0.789487


In [41]:
# add another column
df11 = df.copy()
df11["E"] = ["one", "two", "three", "four",
             "two", "five", "one", "two", "three", "four"]
df11

,A,B,C,D,E
2022-06-19,0.362133,-1.024067,0.443101,0.537134,one
2022-06-20,0.371088,0.729463,-0.953156,1.081669,two
2022-06-21,1.209084,0.286537,0.601964,-0.762744,three
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,four
2022-06-23,0.467150,0.600479,-0.082923,0.051940,two
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571,five
2022-06-25,-0.172528,0.830876,0.912158,1.329058,one
2022-06-26,0.627848,0.435499,2.178048,1.557753,two
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831,three
2022-06-28,-0.436952,1.171574,-0.445353,0.789487,four


In [42]:
# the isin() query - basically the in clause
df11[df11["E"].isin(["two", "five"])]

,A,B,C,D,E
2022-06-20,0.371088,0.729463,-0.953156,1.081669,two
2022-06-23,0.467150,0.600479,-0.082923,0.051940,two
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571,five
2022-06-26,0.627848,0.435499,2.178048,1.557753,two


## Setting values

In [43]:
# matching indexes auto-aligns values
s1 = pd.Series(range(11, 21), index=pd.date_range('20220619', periods=10))
s1

2022-06-19    11
2022-06-20    12
2022-06-21    13
2022-06-22    14
2022-06-23    15
2022-06-24    16
2022-06-25    17
2022-06-26    18
2022-06-27    19
2022-06-28    20
Freq: D, dtype: int64

In [44]:
df["F"] = s1
df

,A,B,C,D,F
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11
2022-06-20,0.371088,0.729463,-0.953156,1.081669,12
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14
2022-06-23,0.467150,0.600479,-0.082923,0.051940,15
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571,16
2022-06-25,-0.172528,0.830876,0.912158,1.329058,17
2022-06-26,0.627848,0.435499,2.178048,1.557753,18
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831,19
2022-06-28,-0.436952,1.171574,-0.445353,0.789487,20


In [45]:
# setting values by label and position
# first let's make a quick copy
df12 = df.copy()
df12

,A,B,C,D,F
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11
2022-06-20,0.371088,0.729463,-0.953156,1.081669,12
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14
2022-06-23,0.467150,0.600479,-0.082923,0.051940,15
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571,16
2022-06-25,-0.172528,0.830876,0.912158,1.329058,17
2022-06-26,0.627848,0.435499,2.178048,1.557753,18
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831,19
2022-06-28,-0.436952,1.171574,-0.445353,0.789487,20


In [46]:
# set by label
df12.at[dates[0], "A"] = 0
# set by position
df12.iat[0, 1] = 0
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,0.443101,0.537134,11
2022-06-20,0.371088,0.729463,-0.953156,1.081669,12
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14
2022-06-23,0.467150,0.600479,-0.082923,0.051940,15
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571,16
2022-06-25,-0.172528,0.830876,0.912158,1.329058,17
2022-06-26,0.627848,0.435499,2.178048,1.557753,18
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831,19
2022-06-28,-0.436952,1.171574,-0.445353,0.789487,20


In [47]:
# kinda bigger replacement
df12.loc[:, "D"] = np.array([5]*len(df))
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,0.443101,5,11
2022-06-20,0.371088,0.729463,-0.953156,5,12
2022-06-21,1.209084,0.286537,0.601964,5,13
2022-06-22,2.597306,-0.306801,-0.664610,5,14
2022-06-23,0.467150,0.600479,-0.082923,5,15
2022-06-24,1.583078,-1.854103,-0.660532,5,16
2022-06-25,-0.172528,0.830876,0.912158,5,17
2022-06-26,0.627848,0.435499,2.178048,5,18
2022-06-27,-0.551157,0.069735,-0.328632,5,19
2022-06-28,-0.436952,1.171574,-0.445353,5,20


In [48]:
# setting values using a boolean selection (aka where clause)
df12[df12 > 0] = -df12
df12

,A,B,C,D,F
2022-06-19,0.000000,0.000000,-0.443101,-5,-11
2022-06-20,-0.371088,-0.729463,-0.953156,-5,-12
2022-06-21,-1.209084,-0.286537,-0.601964,-5,-13
2022-06-22,-2.597306,-0.306801,-0.664610,-5,-14
2022-06-23,-0.467150,-0.600479,-0.082923,-5,-15
2022-06-24,-1.583078,-1.854103,-0.660532,-5,-16
2022-06-25,-0.172528,-0.830876,-0.912158,-5,-17
2022-06-26,-0.627848,-0.435499,-2.178048,-5,-18
2022-06-27,-0.551157,-0.069735,-0.328632,-5,-19
2022-06-28,-0.436952,-1.171574,-0.445353,-5,-20


# Missing Data

### _reindex_
change/add/delete index on a specified axis, returns a new dataframe

In [49]:
df13 = df.reindex(index=dates[0:4], columns=list(df.columns)+["G"])
df13

,A,B,C,D,F,G
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11,NaN
2022-06-20,0.371088,0.729463,-0.953156,1.081669,12,NaN
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,NaN
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,NaN


### handling missing data
1. _```np.nan```_
1. _```pandas.isna()```_
1. ```df.dropna()```
1. ```df.fillna()```

In [50]:
# missing data in pandas is np.nan
df13.iat[1, 0] = np.nan
df13.iloc[1:, 5] = np.random.randint(1)
df13

,A,B,C,D,F,G
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11,NaN
2022-06-20,NaN,0.729463,-0.953156,1.081669,12,0.0
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,0.0
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,0.0


In [51]:
# get a boolean mask where values are NaN
df131 = pd.isna(df13)
df131

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [52]:
# or just
pd.isna(df13)

,A,B,C,D,F,G
2022-06-19,False,False,False,False,False,True
2022-06-20,True,False,False,False,False,False
2022-06-21,False,False,False,False,False,False
2022-06-22,False,False,False,False,False,False


In [53]:
# the original is still there
df13

,A,B,C,D,F,G
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11,NaN
2022-06-20,NaN,0.729463,-0.953156,1.081669,12,0.0
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,0.0
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,0.0


In [54]:
# we are going to drop / replace values now, let's make a couple of copies of the dataframe
df132 = df13.copy()
df132

,A,B,C,D,F,G
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11,NaN
2022-06-20,NaN,0.729463,-0.953156,1.081669,12,0.0
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,0.0
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,0.0


In [55]:
# drop rows/columns that have missing data
# by default it returns a new dataframe, you may want to specify inplace=True for modifying current dataframe:
df_no_na = df132.dropna(how="any")
# how=‘any’ : If any NA values are present, drop that row or column.
# how=‘all’ : If all values are NA, drop that row or column.

In [56]:
# all rows/cols with missing data stripped
df_no_na

,A,B,C,D,F,G
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,0.0
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,0.0


In [57]:
# original still intact
df132

,A,B,C,D,F,G
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11,NaN
2022-06-20,NaN,0.729463,-0.953156,1.081669,12,0.0
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,0.0
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,0.0


In [58]:
# drop missing data from original
df132.dropna(how="any", inplace=True)
df132

,A,B,C,D,F,G
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,0.0
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,0.0


In [59]:
# fill missing data
df133 = df13.copy()
df133.fillna(np.pi*1000)

,A,B,C,D,F,G
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11,3141.592654
2022-06-20,3141.592654,0.729463,-0.953156,1.081669,12,0.000000
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13,0.000000
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14,0.000000


# Align and Join

There needs to be a bigger notebook for this topic.
You need to know there's "joins" in Pandas just like in the SQL world, 
like join and left join and right join and inner and outer and all that...
Here we'll explore these in the context of the align function - 'aligning' the indexes of two dataframes.
Later we'll see these again in the context of _merging_ two data frames.

## Create two datasets with _mismatched_ indexes

In [60]:
# date range indexes
idx1 = pd.date_range('2022-01-01',periods = 10)
# 2022-01-01', '2022-01-02' don't exist in idx2
# '2022-01-11', '2022-01-12' don't exist in idx1
idx2 = pd.date_range('2022-01-03',periods = 10)

In [61]:
# dataframes from indexes
d1 = pd.DataFrame(index = idx1, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

d2 = pd.DataFrame(index = idx2, data = 
    {
        'A': np.random.rand(10),
        'B': np.random.randint(1, high=25, size = 10)
    }
)

## Quick Aside: Rendering two dataframes side-by-side

The trick of rendering 2 dataframes side-by-side was from [this](https://stackoverflow.com/questions/38783027/jupyter-notebook-display-two-pandas-tables-side-by-side) stackoverflow question.

In [62]:
# we gon need to display both data frames side by side, so...
from IPython.display import display_html 
def render_df_side_by_side(a, b, a_title = "", b_title = ""):
    a_styler = a.style.set_table_attributes("style='display:inline'").set_caption(a_title)
    b_styler = b.style.set_table_attributes("style='display:inline'").set_caption(b_title)
    display_html(a_styler._repr_html_()+b_styler._repr_html_(), raw=True)

In [63]:
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.747925,4
2022-01-02 00:00:00,0.344225,10
2022-01-03 00:00:00,0.818474,19
2022-01-04 00:00:00,0.591370,22
2022-01-05 00:00:00,0.682445,20
2022-01-06 00:00:00,0.135814,16
2022-01-07 00:00:00,0.618605,1
2022-01-08 00:00:00,0.715176,7
2022-01-09 00:00:00,0.964975,12
2022-01-10 00:00:00,0.935839,15


Table rendering/styling options is [a bigger discussion](https://pandas.pydata.org/pandas-docs/stable/user_guide/style.html#) to be tackled later.

## .add, .sub, .mul, .div etc.

flexible wrappers (```add, sub, mul, div, mod, pow```) to arithmetic operators: ```+, -, *, /, //, %, **```

In [64]:
# the +, -, *, /, //, %, ** operations align indexes to create a UNION of indexes
d3 = d1+d2
d4 = d1.sub(d2) #equivalent to d1-d2
d5 = d1*d2
d6 = d1.div(d2)

In [65]:
# indexs not present in the other dataframe get a NaN
d3.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,1.726171,22.000000
2022-01-04 00:00:00,0.611078,46.000000
2022-01-05 00:00:00,1.096777,30.000000
2022-01-06 00:00:00,0.185944,25.000000
2022-01-07 00:00:00,0.800848,4.000000
2022-01-08 00:00:00,0.759611,30.000000
2022-01-09 00:00:00,1.774221,32.000000
2022-01-10 00:00:00,1.595985,16.000000


In [66]:
# sub
d4.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,-0.089223,16.000000
2022-01-04 00:00:00,0.571661,-2.000000
2022-01-05 00:00:00,0.268113,10.000000
2022-01-06 00:00:00,0.085685,7.000000
2022-01-07 00:00:00,0.436363,-2.000000
2022-01-08 00:00:00,0.670740,-16.000000
2022-01-09 00:00:00,0.155730,-8.000000
2022-01-10 00:00:00,0.275693,14.000000


In [67]:
# mul
d5.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.742926,57.000000
2022-01-04 00:00:00,0.011655,528.000000
2022-01-05 00:00:00,0.282759,200.000000
2022-01-06 00:00:00,0.006808,144.000000
2022-01-07 00:00:00,0.112736,3.000000
2022-01-08 00:00:00,0.031779,161.000000
2022-01-09 00:00:00,0.780902,240.000000
2022-01-10 00:00:00,0.617791,15.000000


In [68]:
# div
d6.style.highlight_null(null_color='red')

,A,B
2022-01-01 00:00:00,nan,nan
2022-01-02 00:00:00,nan,nan
2022-01-03 00:00:00,0.901704,6.333333
2022-01-04 00:00:00,30.005303,0.916667
2022-01-05 00:00:00,1.647098,2.000000
2022-01-06 00:00:00,2.709258,1.777778
2022-01-07 00:00:00,3.394408,0.333333
2022-01-08 00:00:00,16.094535,0.304348
2022-01-09 00:00:00,1.192438,0.600000
2022-01-10 00:00:00,1.417624,15.000000


### removing the NaN values from the resultant dataframe

In [69]:
# the + operation is UNION of indexes
d31 = d1+d2
# indexs not present in the other dataframe get a NaN
# do you can obvs remove nan values like before
d3.dropna().style.highlight_null(null_color='red')

,A,B
2022-01-03 00:00:00,1.726171,22.000000
2022-01-04 00:00:00,0.611078,46.000000
2022-01-05 00:00:00,1.096777,30.000000
2022-01-06 00:00:00,0.185944,25.000000
2022-01-07 00:00:00,0.800848,4.000000
2022-01-08 00:00:00,0.759611,30.000000
2022-01-09 00:00:00,1.774221,32.000000
2022-01-10 00:00:00,1.595985,16.000000


## df.align()

Pandas [align](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.align.html) works to join the dataframes in interesting ways.

In [70]:
# align returns two dataframes - left and right as a result of a join
# default join is outer - the union of indices will be used as index for left and right
left, right = d1.align(d2, join='outer')
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.747925,4.000000
2022-01-02 00:00:00,0.344225,10.000000
2022-01-03 00:00:00,0.818474,19.000000
2022-01-04 00:00:00,0.591370,22.000000
2022-01-05 00:00:00,0.682445,20.000000
2022-01-06 00:00:00,0.135814,16.000000
2022-01-07 00:00:00,0.618605,1.000000
2022-01-08 00:00:00,0.715176,7.000000
2022-01-09 00:00:00,0.964975,12.000000
2022-01-10 00:00:00,0.935839,15.000000


In [71]:
# inner join
left, right = d1.align(d2, join='inner')
render_df_side_by_side(d1,d2)

,A,B
2022-01-01 00:00:00,0.747925,4
2022-01-02 00:00:00,0.344225,10
2022-01-03 00:00:00,0.818474,19
2022-01-04 00:00:00,0.591370,22
2022-01-05 00:00:00,0.682445,20
2022-01-06 00:00:00,0.135814,16
2022-01-07 00:00:00,0.618605,1
2022-01-08 00:00:00,0.715176,7
2022-01-09 00:00:00,0.964975,12
2022-01-10 00:00:00,0.935839,15


In [72]:
# we can fill the NaNs with a certain value if needed
# let's fill with -100 so we can see it clearly
left, right = d1.align(d2, join='outer', fill_value = -100) 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.747925,4
2022-01-02 00:00:00,0.344225,10
2022-01-03 00:00:00,0.818474,19
2022-01-04 00:00:00,0.591370,22
2022-01-05 00:00:00,0.682445,20
2022-01-06 00:00:00,0.135814,16
2022-01-07 00:00:00,0.618605,1
2022-01-08 00:00:00,0.715176,7
2022-01-09 00:00:00,0.964975,12
2022-01-10 00:00:00,0.935839,15


In [73]:
# left join: left index is added to right, but not the other way
left, right = d1.align(d2, join='left') 
render_df_side_by_side(left, right)

,A,B
2022-01-01 00:00:00,0.747925,4
2022-01-02 00:00:00,0.344225,10
2022-01-03 00:00:00,0.818474,19
2022-01-04 00:00:00,0.591370,22
2022-01-05 00:00:00,0.682445,20
2022-01-06 00:00:00,0.135814,16
2022-01-07 00:00:00,0.618605,1
2022-01-08 00:00:00,0.715176,7
2022-01-09 00:00:00,0.964975,12
2022-01-10 00:00:00,0.935839,15


In [74]:
# right join: right index is added to left, but not the other way
left, right = d1.align(d2, join='right') 
render_df_side_by_side(left, right)

,A,B
2022-01-03 00:00:00,0.818474,19.000000
2022-01-04 00:00:00,0.591370,22.000000
2022-01-05 00:00:00,0.682445,20.000000
2022-01-06 00:00:00,0.135814,16.000000
2022-01-07 00:00:00,0.618605,1.000000
2022-01-08 00:00:00,0.715176,7.000000
2022-01-09 00:00:00,0.964975,12.000000
2022-01-10 00:00:00,0.935839,15.000000
2022-01-11 00:00:00,nan,nan
2022-01-12 00:00:00,nan,nan


There's more to explore in align, and it can get confusing, so beginners be careful, take time and try to work out the result before you execute to build intuition.

# Operations on data

## Stats

Operations in general *exclude* missing data

In [75]:
# arithmetic mean, for each column (axis = 0)
df.mean()

A     0.605705
B     0.093919
C     0.100006
D     0.514741
F    15.500000
dtype: float64

In [76]:
# mean across a row (axis = 1)
df.mean(1)

2022-06-19    2.263660
2022-06-20    2.645813
2022-06-21    2.866968
2022-06-22    3.483082
2022-06-23    3.207329
2022-06-24    2.945174
2022-06-25    3.979913
2022-06-26    4.559829
2022-06-27    3.461223
2022-06-28    4.215751
Freq: D, dtype: float64

## Apply

In [77]:
# apply custom lambdas
df.apply(lambda x: x.max()-x.min())

A    3.148463
B    3.025676
C    3.131204
D    2.673347
F    9.000000
dtype: float64

In [78]:
df

,A,B,C,D,F
2022-06-19,0.362133,-1.024067,0.443101,0.537134,11
2022-06-20,0.371088,0.729463,-0.953156,1.081669,12
2022-06-21,1.209084,0.286537,0.601964,-0.762744,13
2022-06-22,2.597306,-0.306801,-0.664610,1.789516,14
2022-06-23,0.467150,0.600479,-0.082923,0.051940,15
2022-06-24,1.583078,-1.854103,-0.660532,-0.342571,16
2022-06-25,-0.172528,0.830876,0.912158,1.329058,17
2022-06-26,0.627848,0.435499,2.178048,1.557753,18
2022-06-27,-0.551157,0.069735,-0.328632,-0.883831,19
2022-06-28,-0.436952,1.171574,-0.445353,0.789487,20


In [79]:
# each subsequent value is a sum of all values before it in the respective column
cumsum_df = df.apply(np.cumsum)
render_df_side_by_side(df, cumsum_df, "OG Dataframe", "Cumulative Sum")

,A,B,C,D,F
2022-06-19 00:00:00,0.362133,-1.024067,0.443101,0.537134,11
2022-06-20 00:00:00,0.371088,0.729463,-0.953156,1.081669,12
2022-06-21 00:00:00,1.209084,0.286537,0.601964,-0.762744,13
2022-06-22 00:00:00,2.597306,-0.306801,-0.664610,1.789516,14
2022-06-23 00:00:00,0.467150,0.600479,-0.082923,0.051940,15
2022-06-24 00:00:00,1.583078,-1.854103,-0.660532,-0.342571,16
2022-06-25 00:00:00,-0.172528,0.830876,0.912158,1.329058,17
2022-06-26 00:00:00,0.627848,0.435499,2.178048,1.557753,18
2022-06-27 00:00:00,-0.551157,0.069735,-0.328632,-0.883831,19
2022-06-28 00:00:00,-0.436952,1.171574,-0.445353,0.789487,20


## Histogramming

Frequencies, 'nuff said

```value_counts()```

In [80]:
series1 = pd.Series(np.random.randint(0,5, size = 25))
series1.value_counts()

4    7
0    6
2    4
1    4
3    4
dtype: int64

## String Methods

In [81]:
# build a series
some_string = "SERIes and Index are EqUIppeD WITh A seT Of stRINg PROCESSING METHoDs In tHe ```sTR``` attrIBute THAT MakE IT eASy To oPerATE ON EACh ELEmeNt OF The aRrAY. NOtE thAt PAtTERN-maTching in ```sTR``` gENeralLY UsES reGUlAr eXpreSsiONs bY DEfault (aND In Some cases AlwayS uSEs tHeM)."
str_series = pd.Series(some_string.split()) # split splits on whitespace by default

In [82]:
# manipulating the case - lower, upper etc.
# also the length of each string, split, replace, yada yada...
# very important when cleaning data (column names may have stupid whitespace, bad case, spacing etc.)
low = str_series.str.lower()
up = str_series.str.upper()

low_up_df = pd.DataFrame(
    {
        'og': str_series,
        'length': str_series.str.len(),
        'low': low,
        'up': up,
        'split_low_on_i': low.str.split('i', expand = False), # expand = True will break this usecase, try elsewhere
        'replace_a_with_star_in_low': low.str.replace('a','*', case=False, regex=True) # yeah, you can mess with regular expns
    })

low_up_df.tail(10)

,og,length,low,up,split_low_on_i,replace_a_with_star_in_low
36,eXpreSsiONs,11,expressions,EXPRESSIONS,"[express, ons]",expressions
37,bY,2,by,BY,[by],by
38,DEfault,7,default,DEFAULT,[default],def*ult
39,(aND,4,(and,(AND,[(and],(*nd
40,In,2,in,IN,"[, n]",in
41,Some,4,some,SOME,[some],some
42,cases,5,cases,CASES,[cases],c*ses
43,AlwayS,6,always,ALWAYS,[always],*lw*ys
44,uSEs,4,uses,USES,[uses],uses
45,tHeM).,6,them).,THEM).,[them).],them).


See [here](https://pandas.pydata.org/pandas-docs/stable/user_guide/text.html#text-string-methods) for more string operations.

# Merge

Combine series and dataframes, use set logic for indexes and relational algebra for joins/merges

## Concat

Chill, we've done this before. The same join = inner/outer/left/right drill. 

In [83]:
# let's make some more dataframes
concatDF = pd.DataFrame(np.random.randn(10,4)) # 10 rows and 4 columns of random numbers
pieces = [concatDF[:3], concatDF[3:7], concatDF[7:]]
reconstructedDF = pd.concat(pieces)
display('broken DFs: ',pieces)
render_df_side_by_side(concatDF, reconstructedDF, "OG", "Concatenated")

'broken DFs: '

[          0         1         2         3
 0  0.122916 -0.155954  0.292003  0.403000
 1 -1.989882  1.106180  0.959255 -0.669173
 2 -1.805280 -1.480898 -0.622197  0.333966,
           0         1         2         3
 3  0.342642 -0.655954  1.969446  0.593602
 4 -0.242920 -0.083115  1.528123 -1.518348
 5  0.387142  1.379266 -1.601994  1.219840
 6 -0.363508  1.194338 -0.404770  0.067415,
           0         1         2         3
 7  0.522848 -0.341494 -1.139231 -1.616450
 8  1.574348 -1.393101 -2.206774  0.198336
 9 -0.174837  0.135194 -0.081274 -0.101981]

,0,1,2,3
0,0.122916,-0.155954,0.292003,0.403000
1,-1.989882,1.106180,0.959255,-0.669173
2,-1.805280,-1.480898,-0.622197,0.333966
3,0.342642,-0.655954,1.969446,0.593602
4,-0.242920,-0.083115,1.528123,-1.518348
5,0.387142,1.379266,-1.601994,1.219840
6,-0.363508,1.194338,-0.404770,0.067415
7,0.522848,-0.341494,-1.139231,-1.616450
8,1.574348,-1.393101,-2.206774,0.198336
9,-0.174837,0.135194,-0.081274,-0.101981


_From the [page](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html#concat)_:
Adding a column to a DataFrame is relatively fast. However, adding a row requires a copy, and may be expensive. We recommend passing a pre-built list of records to the DataFrame constructor instead of building a DataFrame by iteratively appending records to it.

## Join
SQL style merges. 

In [84]:
# two data frames
sl = pd.Series(np.random.randint(10, size=6))
sr = sl.shift(2).fillna(0).astype(int)
left = pd.DataFrame(
    {
        'key': list('ABCDEF'),
        'lval': sl
    }
)

right = pd.DataFrame(
    {
        'key': list('ABBCDE'),
        'rval': sr
    }
)

In [85]:
render_df_side_by_side(left, right, 'left', 'right')

,key,lval
0,A,1
1,B,9
2,C,8
3,D,2
4,E,8
5,F,2
,key,rval
0,A,0
1,B,0
2,B,1


In [86]:
result = pd.merge(left, right, on='key')
result

,key,lval,rval
0,A,1,0
1,B,9,0
2,B,9,1
3,C,8,9
4,D,2,8
5,E,8,2


# Grouping

"Group By" really means one or more of the following happening:
1. **Split**: data is broken into groups based on some criteria
2. **Apply**: a data operation is executed on each of those groups
3. **Combine**: the results are combined back into a data structure

In [87]:
grp_df = pd.DataFrame(
    {
        'colA': list('AABCAABDAA'),
        'colB': list('1234567890'),
        'colC': np.random.randn(10),
        'colD': np.random.randn(10)
    }
)
grp_df

,colA,colB,colC,colD
0,A,1,1.603506,0.118137
1,A,2,-0.778990,0.069945
2,B,3,0.909510,-0.976162
3,C,4,-1.179740,0.686040
4,A,5,0.477314,-0.279981
5,A,6,-0.143855,-0.835934
6,B,7,-0.926844,-2.236323
7,D,8,-0.224228,1.010345
8,A,9,1.818000,1.137347
9,A,0,-0.483450,-0.604792


In [88]:
# a DataFrameGroupBy object object
grpby = grp_df.groupby('colA')

# list all the groups found
g = grpby.groups
g

{'A': [0, 1, 4, 5, 8, 9], 'B': [2, 6], 'C': [3], 'D': [7]}

In [89]:
# to create a dataframe from the group use something like
grpby.get_group('A')

,colA,colB,colC,colD
0,A,1,1.603506,0.118137
1,A,2,-0.778990,0.069945
4,A,5,0.477314,-0.279981
5,A,6,-0.143855,-0.835934
8,A,9,1.818000,1.137347
9,A,0,-0.483450,-0.604792


### Find all the groups

In [90]:
# but we may not know all the groups in the dataframe.
# so got to find a way to list all groups as dataframes.
# 
# some basic python helps create a list of groupby names
grpby_labels_list = [*g.keys()] # using the unpacking operator *
# alternatively
# grpby_labels_list = list(g.keys())
grpby_labels_list

['A', 'B', 'C', 'D']

In [91]:
# create a dict where the group label is the key and the group dataframe is the value
grps = {lbl: grpby.get_group(lbl) for lbl in grpby_labels_list}

# now use the dict to access individual dataframes created out of the groupby operation
grps['B']

# question: can we create a dataframe of dataframes?

,colA,colB,colC,colD
2,B,3,0.909510,-0.976162
6,B,7,-0.926844,-2.236323


In [92]:
# You can see the datatype of the value is a pandas dataframe
type(grps['A'])

pandas.core.frame.DataFrame

In [93]:
# make a function of it all...
def get_all_groups(df_grpby):
    return {lbl: df_grpby.get_group(lbl) for lbl in [*df_grpby.groups.keys()]}

In [94]:
grps = get_all_groups(grpby)
grps

{'A':   colA colB      colC      colD
 0    A    1  1.603506  0.118137
 1    A    2 -0.778990  0.069945
 4    A    5  0.477314 -0.279981
 5    A    6 -0.143855 -0.835934
 8    A    9  1.818000  1.137347
 9    A    0 -0.483450 -0.604792,
 'B':   colA colB      colC      colD
 2    B    3  0.909510 -0.976162
 6    B    7 -0.926844 -2.236323,
 'C':   colA colB     colC     colD
 3    C    4 -1.17974  0.68604,
 'D':   colA colB      colC      colD
 7    D    8 -0.224228  1.010345}

In [95]:
# get a sum of values from each group
grp_df.groupby('colA').sum()

,colC,colD
colA,,
A,2.492525,-0.395278
B,-0.017334,-3.212485
C,-1.179740,0.686040
D,-0.224228,1.010345


In [96]:
# let's add one more column to the dataframe
# we can try grouping on two columns
grp_df['colE'] = list('1231231231')
grp_df

,colA,colB,colC,colD,colE
0,A,1,1.603506,0.118137,1
1,A,2,-0.778990,0.069945,2
2,B,3,0.909510,-0.976162,3
3,C,4,-1.179740,0.686040,1
4,A,5,0.477314,-0.279981,2
5,A,6,-0.143855,-0.835934,3
6,B,7,-0.926844,-2.236323,1
7,D,8,-0.224228,1.010345,2
8,A,9,1.818000,1.137347,3
9,A,0,-0.483450,-0.604792,1


In [97]:
g2 = grp_df.groupby(['colA','colE'])
grps2 = get_all_groups(g2)
grps2

{('A',
  '1'):   colA colB      colC      colD colE
 0    A    1  1.603506  0.118137    1
 9    A    0 -0.483450 -0.604792    1,
 ('A',
  '2'):   colA colB      colC      colD colE
 1    A    2 -0.778990  0.069945    2
 4    A    5  0.477314 -0.279981    2,
 ('A',
  '3'):   colA colB      colC      colD colE
 5    A    6 -0.143855 -0.835934    3
 8    A    9  1.818000  1.137347    3,
 ('B',
  '1'):   colA colB      colC      colD colE
 6    B    7 -0.926844 -2.236323    1,
 ('B',
  '3'):   colA colB     colC      colD colE
 2    B    3  0.90951 -0.976162    3,
 ('C',
  '1'):   colA colB     colC     colD colE
 3    C    4 -1.17974  0.68604    1,
 ('D',
  '2'):   colA colB      colC      colD colE
 7    D    8 -0.224228  1.010345    2}

In [98]:
render_df_side_by_side(grps2[('A','1')], grps2[('A','2')], 'A,1', 'A,2')

,colA,colB,colC,colD,colE
0,A,1,1.603506,0.118137,1
9,A,0,-0.483450,-0.604792,1
,colA,colB,colC,colD,colE
1,A,2,-0.778990,0.069945,2
4,A,5,0.477314,-0.279981,2


# Reshaping

Stacking and Pivot Tables.
This really warrants more practice, at least go through [the page](https://pandas.pydata.org/pandas-docs/stable/user_guide/reshaping.html#)

## Stacking

In [99]:
tuples = list(
    zip(
        *[
            ["bar", "bar", "baz", "baz", "foo", "foo", "qux", "qux"],
            ["one", "two", "one", "two", "one", "two", "one", "two"],
        ]
    )
)

If the syntax [*expression](https://docs.python.org/dev/reference/expressions.html#calls) appears in the function call, expression must evaluate to an iterable. Elements from these iterables are treated as if they were additional positional arguments.

...although the *expression syntax may appear after explicit keyword arguments, it is processed before the keyword arguments (and any **expression arguments)

In [100]:
# starred expressions...
def f(a,b):
    print(a, ' ', b)
    
f (b=1, *(2,)) # works, the tuple (iterable) is expanded, 2 is assigned to a, then b fills the next argument slot
# f (a=1, *(2,)) # FAILS, multiple values for argument 'a'

2   1


In [101]:
tuples

[('bar', 'one'),
 ('bar', 'two'),
 ('baz', 'one'),
 ('baz', 'two'),
 ('foo', 'one'),
 ('foo', 'two'),
 ('qux', 'one'),
 ('qux', 'two')]

In [102]:
stack_idx = pd.MultiIndex.from_tuples(tuples, names = ['first', 'second'])

In [103]:
stack_df = pd.DataFrame(np.random.randn(8,2), index = stack_idx, columns = ['a','b'])
stack_df

a         b
first second                    
bar   one    -0.645412  0.450751
      two     0.514209 -1.339870
baz   one    -0.424108  0.930662
      two     2.417725  0.990826
foo   one     0.731846 -0.187483
      two    -0.911032  3.313044
qux   one    -0.763669 -0.141793
      two    -0.169546 -0.140653

In [104]:
# stack really just combines the columns
stacked = stack_df.stack()
stacked

first  second   
bar    one     a   -0.645412
               b    0.450751
       two     a    0.514209
               b   -1.339870
baz    one     a   -0.424108
               b    0.930662
       two     a    2.417725
               b    0.990826
foo    one     a    0.731846
               b   -0.187483
       two     a   -0.911032
               b    3.313044
qux    one     a   -0.763669
               b   -0.141793
       two     a   -0.169546
               b   -0.140653
dtype: float64

In [105]:
# unstacked brings back the two columns
# by default unstacks the last level
unstacked = stacked.unstack()
unstacked

a         b
first second                    
bar   one    -0.645412  0.450751
      two     0.514209 -1.339870
baz   one    -0.424108  0.930662
      two     2.417725  0.990826
foo   one     0.731846 -0.187483
      two    -0.911032  3.313044
qux   one    -0.763669 -0.141793
      two    -0.169546 -0.140653

In [106]:
stacked.unstack(1)

second        one       two
first                      
bar   a -0.645412  0.514209
      b  0.450751 -1.339870
baz   a -0.424108  2.417725
      b  0.930662  0.990826
foo   a  0.731846 -0.911032
      b -0.187483  3.313044
qux   a -0.763669 -0.169546
      b -0.141793 -0.140653

In [107]:
stacked.unstack(0)

first          bar       baz       foo       qux
second                                          
one    a -0.645412 -0.424108  0.731846 -0.763669
       b  0.450751  0.930662 -0.187483 -0.141793
two    a  0.514209  2.417725 -0.911032 -0.169546
       b -1.339870  0.990826  3.313044 -0.140653

In [108]:
# you can basically pivot the columns and rows using a combination of stack() and unstack()
unstacked.unstack()

a                   b          
second       one       two       one       two
first                                         
bar    -0.645412  0.514209  0.450751 -1.339870
baz    -0.424108  2.417725  0.930662  0.990826
foo     0.731846 -0.911032 -0.187483  3.313044
qux    -0.763669 -0.169546 -0.141793 -0.140653

## Pivot Tables

In [109]:
# we're now just using data created in the pandas documentation - just cause, time.
pivot_tables_df = pd.DataFrame(
    {
        "A": ["one", "one", "two", "three"] * 3,
        "B": ["A", "B", "C"] * 4,
        "C": ["foo", "foo", "foo", "bar", "bar", "bar"] * 2,
        "D": np.random.randn(12),
        "E": np.random.randn(12),
    }
)
pivot_tables_df

,A,B,C,D,E
0,one,A,foo,-1.267218,-1.042328
1,one,B,foo,-2.849428,-1.401833
2,two,C,foo,-0.092405,1.441885
3,three,A,bar,0.034118,0.864811
4,one,B,bar,-0.064305,-0.685937
5,one,C,bar,-0.552752,-0.657092
6,two,A,foo,0.559493,0.561481
7,three,B,foo,-0.582036,-1.911104
8,one,C,foo,-0.484431,-0.584123
9,one,A,bar,-0.215903,-0.800482


In [110]:
# simple pivot tables - data, values to aggregate, index, columns to pivot along
pd.pivot_table(pivot_tables_df, values = 'D', index = ['A', 'B'], columns = ['C'])

C             bar       foo
A     B                    
one   A -0.215903 -1.267218
      B -0.064305 -2.849428
      C -0.552752 -0.484431
three A  0.034118       NaN
      B       NaN -0.582036
      C -0.800872       NaN
two   A       NaN  0.559493
      B -0.243822       NaN
      C       NaN -0.092405

In [111]:
# more'n one columns of values
# mark all NaN as zero (0)
pd.pivot_table(pivot_tables_df, values = ['D', 'E'], index = ['A', 'B'], columns = ['C']).fillna(0)

D                   E          
C             bar       foo       bar       foo
A     B                                        
one   A -0.215903 -1.267218 -0.800482 -1.042328
      B -0.064305 -2.849428 -0.685937 -1.401833
      C -0.552752 -0.484431 -0.657092 -0.584123
three A  0.034118  0.000000  0.864811  0.000000
      B  0.000000 -0.582036  0.000000 -1.911104
      C -0.800872  0.000000  0.742152  0.000000
two   A  0.000000  0.559493  0.000000  0.561481
      B -0.243822  0.000000  1.918200  0.000000
      C  0.000000 -0.092405  0.000000  1.441885

# Time Series

## Resample
We may need to resample data to a different time-frequency. 
Like get data every second and resample to data every 5 minutes and so on...

In [112]:
rng = pd.date_range('01/01/2022', periods = 1000, freq = 'S')

In [113]:
ts = pd.Series(np.random.randint(0,500, len(rng)), index=rng)

In [114]:
ts.resample('5min').sum()

2022-01-01 00:00:00    76141
2022-01-01 00:05:00    77759
2022-01-01 00:10:00    74504
2022-01-01 00:15:00    23813
Freq: 5T, dtype: int32

read up on [date offset strings](https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html#dateoffset-objects) like '5min' etc.

## Timezones

In [115]:
# timezone representation
rng2 = pd.date_range('01/01/2022', periods = 500, freq='D')
ts2 = pd.Series(np.random.randint(0,365, len(rng2)), index=rng2)
ts2_utc = ts2.tz_localize('UTC')

In [116]:
# convert to US/Eastern
ts2_utc.tz_convert('US/Eastern')

2021-12-31 19:00:00-05:00    173
2022-01-01 19:00:00-05:00      1
2022-01-02 19:00:00-05:00    343
2022-01-03 19:00:00-05:00    138
2022-01-04 19:00:00-05:00    230
                            ... 
2023-05-10 20:00:00-04:00     78
2023-05-11 20:00:00-04:00     35
2023-05-12 20:00:00-04:00     43
2023-05-13 20:00:00-04:00    121
2023-05-14 20:00:00-04:00     18
Freq: D, Length: 500, dtype: int32

## Time span representations

Converting between periods and timestamps.

Period is a _timespan_ - span of time between two timestamps.

In [117]:
# month - end of each 
rng3 = pd.date_range('01/01/2022', periods = 12, freq='M')
rng3

DatetimeIndex(['2022-01-31', '2022-02-28', '2022-03-31', '2022-04-30',
               '2022-05-31', '2022-06-30', '2022-07-31', '2022-08-31',
               '2022-09-30', '2022-10-31', '2022-11-30', '2022-12-31'],
              dtype='datetime64[ns]', freq='M')

In [120]:
ts3 = pd.Series(np.random.randn(len(rng3)), index = rng3)
ts3

2022-01-31   -0.219180
2022-02-28    0.314895
2022-03-31    0.175247
2022-04-30   -0.126694
2022-05-31   -0.142170
2022-06-30   -0.109954
2022-07-31   -0.102932
2022-08-31    0.992888
2022-09-30   -0.420199
2022-10-31    0.396976
2022-11-30   -0.197992
2022-12-31   -0.267047
Freq: M, dtype: float64

In [121]:
ps = ts3.to_period()
ps

2022-01   -0.219180
2022-02    0.314895
2022-03    0.175247
2022-04   -0.126694
2022-05   -0.142170
2022-06   -0.109954
2022-07   -0.102932
2022-08    0.992888
2022-09   -0.420199
2022-10    0.396976
2022-11   -0.197992
2022-12   -0.267047
Freq: M, dtype: float64

In [122]:
ps.to_timestamp()

2022-01-01   -0.219180
2022-02-01    0.314895
2022-03-01    0.175247
2022-04-01   -0.126694
2022-05-01   -0.142170
2022-06-01   -0.109954
2022-07-01   -0.102932
2022-08-01    0.992888
2022-09-01   -0.420199
2022-10-01    0.396976
2022-11-01   -0.197992
2022-12-01   -0.267047
Freq: MS, dtype: float64

A more complex use-case:
convert a _quarterly frequency_ with _year ending in November_ to _9am_ of _the end of the month_ _following_ the _quarter end_